In [1]:
# -*- coding: utf-8 -*-


import numpy as np

import pandas as pd
import tensorflow as tf    
import matplotlib.pyplot as plt

# %matplotlib inline
import matplotlib.image as mpimg
import os
import random
from pathlib import Path


#from PIL import Image
import PIL.Image as Image


from __future__ import print_function



import keras
from keras import Sequential,layers
from keras.layers  import Dense,Dropout,Conv2D,MaxPooling2D,Flatten,GlobalAveragePooling2D,Activation
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.activations import relu,softmax
from keras.preprocessing.image import ImageDataGenerator
from keras import backend as K
from keras.optimizers import SGD, Adam
from keras.models import load_model


#for one hot encoding
from sklearn.preprocessing import LabelEncoder
from keras.utils import to_categorical
from keras.utils import np_utils


# fix random seed for reproducibility 
seed = 7 
np.random.seed(seed)





# Train images and labels processing

# test images and labels processing

"""There are some steps to be performed berfoe CNN

Standardize the images like / = 255
split into train and test validators
build cnn model

Input shape : 
4D tensor with shape: (batch, channels, rows, cols) if data_format is "channels_first" or 
4D tensor with shape: (batch, rows, cols, channels) if data_format is "channels_last".

If you never set data_format, = "channels_first" or "channels_last" then by default it is "channels_last"


Only the first layer will have input shape



This is a "multi-class softmax classification"
"""
print("all imports done")



Using TensorFlow backend.


all imports done


In [2]:
train_dir = 'C:/Users/susarlas/Desktop/poc/citx/train'
train_images = []
train_labels = []
train_data = []



test_dir  = 'C:/Users/susarlas/Desktop/poc/citx/test'
test_images = []
test_labels = []
test_data = []


In [3]:

side = 48
batch_size = 100

img_size = 48
num_channels = 3
img_size_flat = img_size * img_size * num_channels
img_shape = (img_size, img_size)
fc_size=11 #size of the output of final FC layer

img_width  = 48
img_height = 48

In [4]:
# Train Test Image Labelling and Resizing


# Train images and labels processing

for file_name in os.listdir(train_dir):
    img = Image.open(os.path.join(train_dir, file_name))
    
    
    
    # resize the image read
    img = img.resize((side,side), Image.ANTIALIAS) # resize images 
    
    
    
    
    # Reading as an Array to represent Neural Networks
    
    train_images.append(np.asarray(img))  # read image as np array into train_images
#     train_images.append(img)  # read image as np array into train_images
    
    # Train Labels 
    
    if file_name.endswith(".jpg"):
        temp_label = file_name.split('.jpg')
        temp_file_name = str(temp_label[0])
        
    if len(temp_file_name) > 0:
            temp_label_2 = temp_file_name.split('_')
            train_labels.append(temp_label_2[1])

                
encoder = LabelEncoder()
encoder.fit(train_labels) 
encoded_train_labels = encoder.transform(train_labels)
train_labels = np_utils.to_categorical(encoded_train_labels)



In [5]:
# Train images and labels processing
for file_name in os.listdir(test_dir):
    img = Image.open(os.path.join(test_dir, file_name))
    
    
    
    # resize the image read
    img = img.resize((side,side), Image.ANTIALIAS) # resize images 
    
    
    
    
    # Reading as an Array to represent Neural Networks
    
    test_images.append(np.asarray(img))  # read image as np array into train_images

    
    # test Labels 
    
    if file_name.endswith(".jpg"):
        temp_label = file_name.split('.jpg')
        temp_file_name = str(temp_label[0])
        
    if len(temp_file_name) > 0:
            temp_label_2 = temp_file_name.split('_')
            test_labels.append(temp_label_2[1])

                
encoder = LabelEncoder()
encoder.fit(test_labels) 
encoded_test_labels = encoder.transform(test_labels)
test_labels = np_utils.to_categorical(encoded_test_labels)



In [6]:
# Train Test Splitting
X_train = np.asarray(train_images) # i have already converted as array in the above lines of code
y_train = np.array(train_labels)

X_test  = np.asarray(test_images) # i have already converted as array in the above lines of code
y_test  = np.array(test_labels)


print('length of train image set',len(X_train))
print('X_data shape:', X_train[0].shape)
print('y_data shape:', y_train.shape)


print('length of test image set',len(X_test))
print('X_data shape:', X_test[0].shape)
print('y_data shape:', y_test.shape)


length of train image set 310
X_data shape: (48, 48, 3)
y_data shape: (310, 11)
length of test image set 86
X_data shape: (48, 48, 3)
y_data shape: (86, 11)


In [9]:

tf.reset_default_graph()


print('inside')
# Initialising the CNN


if K.image_data_format() == 'channels_first':
    input_shape1 = (3, img_width, img_height)
else:
    input_shape1 = (img_width, img_height, 3)



model = Sequential()

# input
model.add(Conv2D(32, kernel_size=(3, 3), input_shape = input_shape1,  padding="same",activation = 'relu'))
model.add(Dropout(0.2))




#layer1
model.add(Conv2D(filters=32, kernel_size=(3,3),strides=(1, 1),activation='relu',padding="same"))
model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2)))
model.add(Dropout(0.2))

#layer2
model.add(Conv2D(filters=64, kernel_size=(3,3),strides=(1, 1),activation='relu',padding="same"))
model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2)))
model.add(Dropout(0.2))


# Flattening
model.add(Flatten())


#Dense Layer1
model.add(Dense(832, activation='relu'))
model.add(Dropout(0.4))



# dense layer whose output size is fixed in the hyper parameter: fc_size=11
# model.add(Dense(units = 11,activation = softmax(y_train,axis=-1)))

model.add(Dense(units = 11,activation = (tf.nn.softmax)))


# sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9)
sgd = SGD(lr=0.001, decay=1e-6, momentum=0.9, nesterov=True)
optimizer_function = sgd

# Compiling the CNN
model.compile(optimizer = optimizer_function, loss = 'categorical_crossentropy', metrics = ['accuracy'])
print("done with layers")

inside
done with layers


In [10]:
# #Standardization Process : Actually we standardize using the max dimension for each image. Here i used a different approach by taking the highest 255.

# print("before standardization")
X_train = np.array((X_train/255.0),dtype=np.float32)
X_test = np.array((X_test/255.0), dtype=np.float32)
# print("after standardization")


In [11]:
# Train Model
print("before history")
history = model.fit(X_train, y_train, validation_data=(X_test,y_test),shuffle=True,epochs=250, batch_size=10,verbose=1)
print('after history')

before history
Train on 310 samples, validate on 86 samples
Epoch 1/250
310/310 [==============================] - 15s 49ms/step - loss: 2.0643 - acc: 0.2484 - val_loss: 2.2465 - val_acc: 0.4419
Epoch 2/250
310/310 [==============================] - 12s 39ms/step - loss: 1.9766 - acc: 0.2548 - val_loss: 2.1217 - val_acc: 0.4419
Epoch 3/250
310/310 [==============================] - 12s 39ms/step - loss: 1.9703 - acc: 0.2806 - val_loss: 2.0824 - val_acc: 0.4419
Epoch 4/250
310/310 [==============================] - 12s 38ms/step - loss: 1.9570 - acc: 0.2581 - val_loss: 2.0859 - val_acc: 0.4419
Epoch 5/250
310/310 [==============================] - 12s 40ms/step - loss: 1.9365 - acc: 0.2645 - val_loss: 2.1059 - val_acc: 0.4419
Epoch 6/250
310/310 [==============================] - 12s 37ms/step - loss: 1.9423 - acc: 0.2677 - val_loss: 2.1905 - val_acc: 0.4419
Epoch 7/250
310/310 [==============================] - 11s 36ms/step - loss: 1.9304 - acc: 0.2806 - val_loss: 2.1863 - val_acc: 0.

In [12]:
# K.clear_session()
# tf.reset_default_graph()
# print(test_labels)

In [13]:
# evaluate the model
accuracy = model.evaluate(x=X_test,y=y_test,batch_size=32)

86/86 [==============================] - 0s 5ms/step


In [14]:

# save model and architecture to single file
model.summary()
from keras.models import load_model
model.save('C:/Users/susarlas/Desktop/poc/citx/my_model_2.hd5')
del model


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 48, 48, 32)        896       
_________________________________________________________________
dropout_1 (Dropout)          (None, 48, 48, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 48, 48, 32)        9248      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 24, 24, 32)        0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 24, 24, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 24, 24, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 12, 12, 64)        0         
__________

In [17]:
K.clear_session()
print("session cleared")


session cleared


In [19]:
new_model = load_model('C:/Users/susarlas/Desktop/poc/citx/my_model_2.h5')
print(new_model.summary())


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 48, 48, 32)        896       
_________________________________________________________________
dropout_1 (Dropout)          (None, 48, 48, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 48, 48, 32)        9248      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 24, 24, 32)        0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 24, 24, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 24, 24, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 12, 12, 64)        0         
__________

In [23]:

# To Reverse Map Categorical values to the One Hot Encoded 

def one_hot_to_indices(data):
    indices = []
    for el in data:
        indices.append(list(el).index(1))
    return indices
indices = one_hot_to_indices(train_labels)
print(indices)

[5, 0, 1, 2, 3, 4, 6, 8, 9, 10, 0, 1, 3, 6, 8, 9, 0, 1, 6, 8, 9, 0, 1, 6, 8, 9, 0, 1, 6, 8, 9, 0, 1, 6, 8, 9, 0, 1, 6, 8, 9, 0, 1, 6, 8, 9, 0, 1, 6, 8, 9, 0, 1, 6, 8, 9, 0, 1, 6, 8, 9, 5, 0, 1, 2, 3, 4, 6, 7, 8, 9, 10, 0, 1, 6, 8, 9, 0, 1, 6, 8, 9, 0, 1, 6, 8, 9, 0, 1, 6, 8, 9, 0, 1, 6, 8, 9, 0, 1, 6, 8, 9, 0, 6, 8, 9, 0, 6, 8, 9, 0, 6, 8, 9, 0, 6, 8, 9, 5, 0, 1, 2, 3, 4, 6, 7, 8, 9, 10, 0, 6, 8, 9, 0, 6, 8, 9, 0, 6, 8, 9, 0, 6, 9, 0, 6, 9, 0, 6, 9, 0, 6, 9, 0, 6, 9, 0, 6, 9, 0, 6, 9, 5, 0, 1, 2, 3, 4, 6, 7, 8, 9, 0, 6, 9, 0, 6, 9, 0, 6, 9, 0, 6, 9, 0, 6, 9, 0, 6, 9, 0, 6, 9, 0, 6, 9, 0, 6, 9, 0, 6, 9, 0, 1, 2, 3, 6, 7, 8, 9, 0, 6, 9, 0, 6, 9, 0, 6, 9, 0, 6, 9, 0, 6, 9, 0, 6, 9, 0, 6, 9, 0, 9, 0, 9, 0, 9, 0, 1, 2, 3, 6, 7, 8, 9, 0, 9, 0, 9, 0, 9, 0, 9, 0, 9, 0, 0, 0, 0, 0, 0, 1, 2, 3, 6, 7, 8, 9, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 2, 3, 6, 7, 8, 9, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 2, 3, 6, 8, 9, 0, 0, 1, 3, 6, 8, 9]
